In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Alpha 15
[nltk_data]     A3DD\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import re
import plotly.express as px
import string as st
import pandas as pd
import numpy as np
import pickle
from nltk.tokenize import word_tokenize, sent_tokenize
import time

# Load data

In [9]:
df_sent = pd.read_csv('prep_text.csv',usecols=['GVKEY','Document','sentence','prep_text'])
df_sent.sample(5)

,GVKEY,Document,sentence,prep_text
9557447,4611,Document DJDN000020210927eh9r001x0,"We celebrate occasions big and small, and have...",We celebrate occasions big and small and have ...
10916234,6266,Document LBA0000020210420eh4k02y91,The EMA examined eight cases of clotting that ...,The EMA examined eight cases of clotting that ...
2187379,14219,Document PRN0000020200122eg1m000ux,-- Generated organic total deposit growth o...,Generated organic total deposit growth of appr...
8567859,3226,Document BWR0000020200430eg4u00067,"Equity in net income (losses) of investees,",Equity in net income losses of investees
6224367,2285,Document DJDN000020200207eg270005v,The Nikkei Stock Average is up 0.2% at 23924.32.,The Nikkei Stock Average is up at


In [6]:
print(df_sent.shape)

(14547874, 4)


# Load model

In [8]:
filename = 'model/3class_LR_model_97acc.sav'
model = pickle.load(open(filename, 'rb'))

# Prediction

In [ ]:
# df_sent['predicted'] = df_sent.prep_text.apply(lambda x : model.predict([x])[0])

In [10]:
# predict cf_class
start = time.time()
df_sent['cf_class'] = df_sent.prep_text.apply(lambda x : model.predict_proba([x])[0])
stop = time.time()
df_sent.sample(5)

,GVKEY,Document,sentence,prep_text,cf_class
12246528,7017,Document LBA0000020200928eg9s00x9a,* Outfront Media Inc : Citigroup r...,Outfront Media Inc Citigroup raises to buy fro...,"[0.0051888780567391915, 0.988553553873082, 0.0..."
11820202,65399,Document PRN0000020200630eg6u000xj,With more than a 110-year legacy of providing ...,With more than a year legacy of providing inno...,"[2.720279512340496e-07, 2.4042283324421104e-09..."
11398679,64768,Document DJDN000020200901eg91001qy,Stay up to date on all things Twitch on Twitte...,Stay up to date on all things Twitch on Twitte...,"[0.022403297767926916, 0.961800271901087, 0.01..."
9432502,4503,Document DJDN000020200921eg9l001ls,"Since then, it has brought on an array of inve...",Since then it has brought on an array of inves...,"[0.009878252617927675, 0.005452207504180924, 0..."
4314151,171007,Document BWR0000020200729eg7t000gk,(2) Guidance for GAAP subscription revenues a...,Guidance for GAAP subscription revenues and no...,"[0.00587588109809505, 0.9849443114287091, 0.00..."


In [11]:
print('Run time :', stop-start, 'sec')

Run time : 305613.4675400257 sec


In [12]:
df_sent.drop('prep_text', axis=1, inplace=True)

In [13]:
# df_sent.to_csv('Fact-result.csv', index=False)

In [14]:
df_sent['neg'] = df_sent.cf_class.apply(lambda x : np.round(x[0],4))
df_sent['neu'] = df_sent.cf_class.apply(lambda x : np.round(x[1],4))
df_sent['pos'] = df_sent.cf_class.apply(lambda x : np.round(x[2],4))

In [15]:
df_sent.sample(5)

,GVKEY,Document,sentence,cf_class,neg,neu,pos
4456682,174389,Document PRN0000020210218eh2i000vp,"Mr. Flanders concluded, ""Over the last four-pl...","[8.379714692325543e-05, 4.298120298028924e-06,...",0.0001,0.0000,0.9999
9063006,3946,Document AWOPEN0020201028egas00209,"changes in cash of assets held for sale, cash ...","[0.013715013761969846, 0.011604153321830025, 0...",0.0137,0.0116,0.9747
4964937,18086,Document PRN0000020210709eh790007i,Information regarding how to assert your Opioi...,"[0.011512518570556319, 0.01063224143064226, 0....",0.0115,0.0106,0.9779
11025223,6304,Document DJDN000020210203eh230049x,"""Our results for the December quarter and cale...","[0.0010802277483768395, 0.00010549365939026858...",0.0011,0.0001,0.9988
8441640,31587,Document DJDN000020210929eh9t001nh,"Dollar Tree, Inc. (NASDAQ: DLTR) today reporte...","[0.00019607354350160335, 4.868881977848404e-06...",0.0002,0.0000,0.9998


In [16]:
df_sent.drop('cf_class', axis=1, inplace=True)

In [17]:
def sel_sentiment(p_neg,p_neu,p_pos):
    if p_neg > p_neu and p_neg > p_pos:
        return 'neg'
    elif p_neu > p_neg and p_neu > p_pos:
        return 'neu'
    elif p_pos > p_neg and p_pos > p_neu:
        return 'pos'

In [21]:
df_sent['predicted'] = df_sent.apply(lambda x : sel_sentiment(x.neg,x.neu,x.pos), axis=1)
df_sent.sample(10)

,GVKEY,Document,sentence,neg,neu,pos,predicted
11429628,64768,Document DJDN000020200730eg7u003wz,-- UNC Health built a chatbot on AWS...,0.0000,0.0000,1.0000,pos
1074019,11896,Document DJDN000020210119eh1j000ua,Note: Unless otherwise mentioned the reporting...,0.0000,0.9275,0.0725,neu
12455574,7257,Document BWR0000020210407eh4700042,IMV Inc. is a clinical stage biopharmaceutical...,0.6617,0.0000,0.3383,neg
9639290,4723,Document DJDN000020201014egae001z5,0837 ET - Next month's U.S. presidential elect...,0.0210,0.0522,0.9269,pos
7555384,28322,Document DJDN000020210721eh7l000xd,13F data provided by: Factset Research Systems...,0.0108,0.9748,0.0144,neu
11172728,63739,Document RTNW000020200311eg3b0017o,Among the restaurateurs trading lower Wednesda...,0.4308,0.0285,0.5408,pos
6622279,24468,Document DJDN000020211020ehak002u7,Wall Street analysts were expecting sales of $...,0.0036,0.0028,0.9935,pos
10354685,5860,Document BWR0000020200221eg2l0004b,Revenue Segment OI Margin ...,0.0014,0.9941,0.0044,neu
6145688,2285,Document RSTPROBK20210504eh54000dx,Write to Aisha Al-Muslim at aisha.al-muslim@ws...,0.0010,0.9980,0.0010,neu
998843,117768,Document DJDN000020210104eh14002h1,"In gaming, NVIDIA's technology leadership has ...",0.0000,0.0000,1.0000,pos


In [ ]:
def enc(text):
    return text.encode('latin-1', 'replace').decode('latin-1')

In [ ]:
df['sentence'] = df.sentence.apply(enc)

In [22]:
# df_sent.to_csv('Fact-sent-result.csv', index=False)

In [ ]:
# df_sent.to_stata('Fact-sent-result.dta', write_index=False, version=117)

# Article Level

In [56]:
def two_sentiment(neg,pos):
    if neg > pos:
        return 'neg'
    else:
        return 'pos'

In [55]:
df_doc = df_sent.groupby(['GVKEY','Document']).agg({'neg':'mean','neu':'mean','pos':'mean'})
df_doc.columns = ['mean_neg','mean_neu', 'mean_pos']
df_doc = df_doc.reset_index()
df_doc

,GVKEY,Document,mean_neg,mean_neu,mean_pos
0,1045,Document ACWIRE0020200727eg7r001p5,0.036158,0.446373,0.517453
1,1045,Document ACWIRE0020210218eh2i002s3,0.053538,0.401670,0.544788
2,1045,Document ACWIRE0020210406eh4600335,0.050700,0.484043,0.465247
3,1045,Document ACWIRE0020210526eh5q0040h,0.045377,0.446490,0.508128
4,1045,Document ACWIRE0020210611eh6a0002t,0.053110,0.414447,0.532443
...,...,...,...,...,...
454043,328795,Document MOODPR0020210329eh3t000rt,0.102557,0.298800,0.598639
454044,328795,Document MOODPR0020210607eh6700135,0.115131,0.262512,0.622367
454045,328795,Document RTNW000020210323eh3n0000m,0.008156,0.635678,0.356144
454046,328795,Document RTNW000020210527eh5r000qu,0.007157,0.145900,0.846929


In [57]:
df_doc['predicted'] = df_doc.apply(lambda x : two_sentiment(x.mean_neg, x.mean_pos), axis=1)
df_doc

,GVKEY,Document,mean_neg,mean_neu,mean_pos,predicted
0,1045,Document ACWIRE0020200727eg7r001p5,0.036158,0.446373,0.517453,pos
1,1045,Document ACWIRE0020210218eh2i002s3,0.053538,0.401670,0.544788,pos
2,1045,Document ACWIRE0020210406eh4600335,0.050700,0.484043,0.465247,pos
3,1045,Document ACWIRE0020210526eh5q0040h,0.045377,0.446490,0.508128,pos
4,1045,Document ACWIRE0020210611eh6a0002t,0.053110,0.414447,0.532443,pos
...,...,...,...,...,...,...
454043,328795,Document MOODPR0020210329eh3t000rt,0.102557,0.298800,0.598639,pos
454044,328795,Document MOODPR0020210607eh6700135,0.115131,0.262512,0.622367,pos
454045,328795,Document RTNW000020210323eh3n0000m,0.008156,0.635678,0.356144,pos
454046,328795,Document RTNW000020210527eh5r000qu,0.007157,0.145900,0.846929,pos


In [64]:
meta = pd.read_csv('unique_data.csv',usecols=['Document','Date','Time','Words'])
meta.head()

,Document,Date,Time,Words
0,Document PRN0000020200417eg4h000es,17 April 2020,18:09,304 words
1,Document PRN0000020200306eg36000ei,6 March 2020,20:21,328 words
2,Document PRN0000020210505eh55000gx,5 May 2021,13:30,"2,145 words"
3,Document PRN0000020210225eh2p0008f,25 February 2021,12:30,370 words
4,Document PRN0000020200513eg5d0009c,13 May 2020,13:30,332 words


In [67]:
join = pd.merge(df_doc, meta, on='Document', how='left')
join.sample(3)

,GVKEY,Document,mean_neg,mean_neu,mean_pos,predicted,Date,Time,Words
41255,2968,Document PRN0000020210504eh540008k,0.066194,0.404557,0.529249,pos,4 May 2021,11:30,"1,354 words"
276542,24405,Document DJDN000020210712eh7c001ym,0.008977,0.298631,0.692397,pos,12 July 2021,13:00,902 words
81071,4839,Document DJDN000020210826eh8q003l9,0.599533,0.038433,0.362033,neg,26 August 2021,21:37,114 words


In [69]:
res = join[['GVKEY','Document','Date','Time','Words','mean_neg','mean_neu','mean_pos','predicted']]
res

,GVKEY,Document,Date,Time,Words,mean_neg,mean_neu,mean_pos,predicted
0,1045,Document ACWIRE0020200727eg7r001p5,27 July 2020,-,"1,181 words",0.036158,0.446373,0.517453,pos
1,1045,Document ACWIRE0020210218eh2i002s3,18 February 2021,-,"1,235 words",0.053538,0.401670,0.544788,pos
2,1045,Document ACWIRE0020210406eh4600335,6 April 2021,-,"1,243 words",0.050700,0.484043,0.465247,pos
3,1045,Document ACWIRE0020210526eh5q0040h,26 May 2021,-,"1,452 words",0.045377,0.446490,0.508128,pos
4,1045,Document ACWIRE0020210611eh6a0002t,10 June 2021,-,"1,229 words",0.053110,0.414447,0.532443,pos
...,...,...,...,...,...,...,...,...,...
454043,328795,Document MOODPR0020210329eh3t000rt,29 March 2021,-,"2,855 words",0.102557,0.298800,0.598639,pos
454044,328795,Document MOODPR0020210607eh6700135,7 June 2021,-,"1,958 words",0.115131,0.262512,0.622367,pos
454045,328795,Document RTNW000020210323eh3n0000m,23 March 2021,01:03,181 words,0.008156,0.635678,0.356144,pos
454046,328795,Document RTNW000020210527eh5r000qu,28 May 2021,00:17,118 words,0.007157,0.145900,0.846929,pos


In [70]:
# res.to_csv('Fact-doc-result.csv', index=False)

In [72]:
# res.to_stata('Fact-doc-result.dta', write_index=False)

In [77]:
# sample
df_sent[:1000].to_csv('Sample-sent-result.csv', index=False)
df_sent[:1000].to_stata('Sample-sent-result.dta', write_index=False, version=117)
res[:1000].to_csv('Sample-doc-result.csv', index=False)
res[:1000].to_stata('Sample-doc-result.dta', write_index=False)